In [1]:
import torch.nn as nn
from torch.nn import functional as F
import torch
from models.digit_classifier import DigitClassifier
from utils import accuracy
from utils import train_model_double_objective
from importlib import reload
from models.net3 import Net3

In [2]:
from dlc_practical_prologue import generate_pair_sets

nSamples = 1000
data = generate_pair_sets(nSamples)
var_names = ["train_input", "train_target", "train_classes", "test_input", "test_target", "test_classes"]
train_input, train_target, train_classes, test_input, test_target, test_classes = data
stringWidth = 20

In [ ]:
#pre-train model for classifying digits
from models.digit_classifier import DigitClassifier
import torch
from torch import nn
from utils import train_model

mini_batch_size = 100
model_classifier = DigitClassifier(out_h = 10, subnet = False)

optimizer = torch.optim.Adam(model_classifier.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()
train_in = train_input[:, 0, :, :].unsqueeze(axis = 1)
train_class = train_classes[:,0]
test_in = test_input[:, 0, :, :].unsqueeze(axis = 1)
test_class = test_classes[:,0]

train_model(model = model_classifier, 
            train_input = train_in, train_target = train_class,
            test_input = test_in, test_target = test_class,
            nb_epochs = 20, mini_batch_size = 100, 
            optimizer = optimizer, criterion = criterion,
            verbose = True)

In [3]:
model = Net3(nb_residual_blocks = 20, model_class_digit, c1 = 32, c2 = 32, c3 = 64, h = 100, p = 0.3, hidden_channel_1 = 1, hidden_channel_2 = 10)
criterion = nn.BCEWithLogitsLoss()
criterion2 = nn.CrossEntropyLoss()
eta = 1e-3
mini_batch_size = 100
optimizer = torch.optim.Adam(model.parameters(), lr = eta)
nb_epochs = 10

train_model_double_objective(model, train_input, train_target, 
                             train_classes, test_input, test_target, test_classes, 
                             nb_epochs, mini_batch_size, optimizer, criterion, criterion2, 
                             beta = 1, verbose = True)

/home/relja/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:145: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(


Epoch 0: loss 78.465 / train accuracy 62.0%, test accuracy 59.6
Epoch 1: loss 45.323 / train accuracy 65.5%, test accuracy 65.6
Epoch 2: loss 36.257 / train accuracy 67.0%, test accuracy 65.5
Epoch 3: loss 28.062 / train accuracy 69.5%, test accuracy 68.1
Epoch 4: loss 21.097 / train accuracy 76.7%, test accuracy 74.2
Epoch 5: loss 17.770 / train accuracy 81.2%, test accuracy 78.6
Epoch 6: loss 15.645 / train accuracy 83.4%, test accuracy 80.4
Epoch 7: loss 12.584 / train accuracy 86.4%, test accuracy 81.0
Epoch 8: loss 11.352 / train accuracy 87.6%, test accuracy 82.9
Epoch 9: loss 9.866 / train accuracy 88.2%, test accuracy 84.3


In [4]:
from utils import accuracy_of_digit_class
print("Accuracy: {:.2f}".format(accuracy_of_digit_class(model, test_input, test_classes)))

Accuracy: 92.20


In [5]:
nParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('The model has {:d} trainable parameters'.format(nParams))

The model has 29605 trainable parameters
